In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.callbacks import TensorBoard
import tensorflow_addons as tfa
import pandas as pd
import numpy as np
from multiprocessing import  Pool
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from time import time

In [2]:
print("open dataset")
train = pd.read_csv("../dataset/goodreads_train.csv")

open dataset


In [3]:
train_prepro = pd.DataFrame(data=np.load(file="../prepro_train_archive_PN_less.npy", allow_pickle=True), columns=['review_text'])['review_text']
# test = pd.read_csv("dataset/goodreads_test.csv")

In [4]:
train['review_text'] = train_prepro

In [5]:
print("0  ",train[train['rating'] == 0]["rating"].count())
print("1  ", train[train['rating'] == 1]["rating"].count())
print("2  ", train[train['rating'] == 2]["rating"].count())
print("3  ", train[train['rating'] == 3]["rating"].count())
print("4  ", train[train['rating'] == 4]["rating"].count())
print("5  ", train[train['rating'] == 5]["rating"].count())
train_3_5 = train[train['rating'] >= 3]
train_3_5

0   30988
1   28718
2   72627
3   188972
4   313688
5   265007


,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,this special book . it started slow first thir...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,recommended . free : http : //www.audible.com/...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,NaN,NaN,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"a fun , fast paced science fiction thriller . ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"i really enjoyed book , lot recommend . it dra...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1
5,8842281e1d1347389f2ab93d60773d4d,19398490,ea4a220b10e6b5c796dae0e3b970aff1,4,a beautiful story . it rare encounter book goo...,Sun Jan 03 21:20:46 -0800 2016,Tue Sep 20 23:30:15 -0700 2016,Tue Sep 13 11:51:51 -0700 2016,Sat Aug 20 07:03:03 -0700 2016,35,5
...,...,...,...,...,...,...,...,...,...,...,...
899995,b9450d1c1f97f891c392b1105959b56e,11832081,7c34c15ad8cb28a4afca32946e2016ea,3,3.5 star . popular author agent want character...,Fri Nov 16 22:10:17 -0800 2012,Sat Nov 17 22:42:44 -0800 2012,Sat Nov 17 00:00:00 -0800 2012,Fri Nov 16 00:00:00 -0800 2012,0,0
899996,b9450d1c1f97f891c392b1105959b56e,16095092,e979c735276431915d089cfaf446d64f,3,this quick read . i read lot book recently one...,Fri Nov 16 22:09:49 -0800 2012,Sat Nov 17 23:38:58 -0800 2012,Sat Nov 17 00:00:00 -0800 2012,Fri Nov 16 00:00:00 -0800 2012,1,1
899997,b9450d1c1f97f891c392b1105959b56e,8430896,feb9fe6439f9fbba8a1cce36020ad977,4,* spoiler alert * 3.5 star . this book sweet i...,Tue Nov 13 05:38:24 -0800 2012,Tue Nov 13 23:29:17 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,1,0
899998,b9450d1c1f97f891c392b1105959b56e,12275680,741e29369f70531bfffbd02add86f228,4,* spoiler alert another fun read ! 's new assi...,Mon Nov 12 05:42:30 -0800 2012,Tue Nov 13 22:32:05 -0800 2012,Tue Nov 13 00:00:00 -0800 2012,Mon Nov 12 00:00:00 -0800 2012,0,0


In [23]:
nb_comments =  train_3_5["n_comments"]
nb_votes = train_3_5["n_votes"]
truc = train_3_5['rating']
type(train['rating'])
type(truc)

pandas.core.series.Series

In [20]:
rating = keras.utils.to_categorical(train['rating'], num_classes=6)

In [ ]:
rating

In [26]:
inputs = keras.Input(shape=(1,), dtype=tf.string)
inputs2 = keras.Input(shape=(1), dtype=tf.int64)
inputs3 = keras.Input(shape=(1), dtype=tf.int64)

vectorize_layer = keras.layers.TextVectorization(
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    output_mode='int',
    output_sequence_length=1400,
    vocabulary=np.load('../vocabulaires/voc_lemm_without_NP.npy')
)(inputs)
conc = keras.layers.concatenate([vectorize_layer, inputs2,inputs3])

layer1 = keras.layers.Dense(100, activation=tf.keras.activations.tanh)(conc)
layer2 = keras.layers.Dense(100, activation=tf.keras.activations.tanh)(layer1)
layer3 = keras.layers.Dense(100, activation=tf.keras.activations.tanh)(layer2)
outputs = keras.layers.Dense(6, activation=tf.keras.activations.tanh)(layer3)

In [27]:
model = keras.Model(inputs=[inputs, inputs2, inputs3], outputs=outputs, name="mnist_model")
tensorboard = TensorBoard(log_dir="../logs/relu".format(time()))

In [28]:
#learning_rates = [0.01, 0.001, 0.000001, 0.0000001]

In [30]:
#for learning_rate in learning_rates:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=[tf.keras.metrics.categorical_accuracy]
              )
model.fit([train_prepro, nb_comments, nb_votes], keras.utils.to_categorical(train['rating'], num_classes=6), epochs=4,
                  callbacks=[
                      tf.keras.callbacks.TensorBoard(log_dir="../logs/relu"),
                  ],
                  batch_size=10000
                  )

Epoch 1/4
90/90 [==============================] - 25s 278ms/step - loss: 1.4690 - categorical_accuracy: 0.2396
Epoch 2/4
90/90 [==============================] - 26s 284ms/step - loss: 1.3613 - categorical_accuracy: 0.2186
Epoch 3/4
90/90 [==============================] - 26s 292ms/step - loss: 1.3345 - categorical_accuracy: 0.1932
Epoch 4/4
90/90 [==============================] - 27s 295ms/step - loss: 1.3169 - categorical_accuracy: 0.2196


In [11]:
#model.save("../models_trained/linear_model_2")

In [15]:
model.predict([train_prepro,nb_comments, nb_votes])

28125/28125 [==============================] - 115s 4ms/step


array([[-0.18595007, -0.8972698 , -0.3929689 ,  0.79503214,  0.74524134,
         0.60949266],
       [ 0.26015648, -0.41109988,  0.10981959,  0.23053342,  0.3345841 ,
         0.3506639 ],
       [ 0.3790108 , -0.87722063, -0.04805964,  0.16727996,  0.55584353,
         0.35468698],
       ...,
       [-0.6583284 , -0.8462807 ,  0.3042675 , -0.20152466,  0.22014128,
         0.43758956],
       [-0.34691152, -0.83316463,  0.60525113,  0.5558279 ,  0.5043566 ,
         0.5192424 ],
       [-0.49532208, -0.8980831 ,  0.39868203,  0.3463079 ,  0.61871976,
         0.48810557]], dtype=float32)